In [1]:
pip install dspy-ai

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\dbake\Documents\GREP_Boi\CS541_CS441_AI_Final_Project\SyntheticData\synth-env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install --upgrade jupyter

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\dbake\Documents\GREP_Boi\CS541_CS441_AI_Final_Project\SyntheticData\synth-env\Scripts\python.exe -m pip install --upgrade pip' command.


### Use DSPy to load CSV Files

In [5]:
import dspy
from dspy.datasets import DataLoader

dl = DataLoader()

In [9]:
grep_options_dataset = dl.from_csv("./data/grep_options.csv",)
sample_files_dataset = dl.from_csv("./data/sample_files.csv",)

In [12]:
print(grep_options_dataset[0])
print(sample_files_dataset[0])

Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'File_Path': '/var/log/syslo?.log', 'Description': 'System log file with one character wildcard'}) (input_keys=set())


### Add sample doc column

In [16]:
# define typed signature
import os

from dotenv import load_dotenv

load_dotenv("../.env")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

gemini_flash = dspy.Google(model="gemini-1.5-flash-latest", temperature=0)

dspy.settings.configure(lm=gemini_flash)

In [76]:
from pydantic import BaseModel, Field, confloat
from typing import List, NamedTuple

class FileData(dspy.Signature):
    path = dspy.InputField(desc="The file, files, or directory")
    description = dspy.InputField(desc="The description of the file, files, or directory")
    file = dspy.OutputField(desc="No more than a 5 line sample of the files based on the description and path. Just include the file. Max 200 tokens")

GenerateSampleFile = dspy.Predict(FileData, temperature=0.6)

In [85]:
resp = GenerateSampleFile(path=sample_files_dataset[4].File_Path, description=sample_files_dataset[4].Description)

In [86]:
resp

Prediction(
    file='```python\ndef main():\n    """Main function for the application."""\n    # ... your code here ...\n\nif __name__ == "__main__":\n    main()\n```'
)

### Create list of files

In [115]:
import time
file_sam = []

for idx, x in enumerate(sample_files_dataset[142:]):
    pred = GenerateSampleFile(path=x.File_Path, description=x.Description)
    file_sam.append(pred.file)    
    print(pred.file)
    time.sleep(10)

```nginx
# This is a comment.
user nginx;
worker_processes auto;
error_log /var/log/nginx/error.log;
pid /run/nginx.pid;

```
```
#!/bin/bash
# This script will print the current date and time.
date

```
```
# This file is part of the GNU C Library.
#
# This library is free software; you can redistribute it and/or
# modify it under the terms of the GNU Lesser General Public
# License as published by the Free Software Foundation; either
# version 2.1 of the License, or (at your option) any later version.
#
# This library is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
# Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with this library; if not, write to the Free Software
# Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301 USA.
```
```
--- a/tmp/te

In [114]:
print(file_sam[25])

```
# Subsystem sftp /usr/lib/openssh/sftp-server
# Port 22
# Protocol 2
# Ciphers aes128-cbc,aes192-cbc,aes256-cbc,arcfour256,arcfour128,blowfish-cbc
# MACs hmac-sha2-256,hmac-sha2-512,hmac-sha1,umac-64@openssh.com,hmac-ripemd160,hmac-md5

```


In [116]:
import csv

csv_file_path = "./data/filetemp5.csv"

# Write the values to the CSV file
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    # Write each value in a new row
    for value in file_sam:
        writer.writerow([value])

In [89]:
import pandas as pd



# Path to the existing CSV file
csv_file_path = "./data/sample_files.csv"

# Read the existing CSV file
df = pd.read_csv(csv_file_path)

# Add the file paths as a new column in the DataFrame
df['file'] = file_sam

# Save the updated DataFrame back to the CSV file
df.to_csv(csv_file_path, index=False)

ValueError: Length of values (44) does not match length of index (150)

### Make metric to choose the best one

In [65]:
class MetricOutput(BaseModel):
    score: confloat(ge=1,le=5) = Field(...,description="Rate the sample of the file given the description")

class MetricSignature(dspy.Signature):
    description = dspy.InputField(desc="The description of what the file is")
    file = dspy.InputField(desc="The file to be judged")
    score: MetricOutput = dspy.OutputField()

get_score = dspy.TypedPredictor(MetricSignature)

#Non-standard implementation of metric
def rate_response(example, pred,trace=None):
    score_response = get_score(description=example.Description, file=pred.file)
    return score_response.score

In [51]:
print(resp.completions.file)

['```\n---\nPath: /var/log/auth.lo?\nDescription: Authentication log file with one character wildcard\nFile: \nFeb 15 10:10:10 server sshd[22345]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=192.168.1.10  user=bob\nFeb 15 10:10:11 server sshd[22345]: Failed password for invalid user bob from 192.168.1.10 port 51050 ssh2\nFeb 15 10:10:12 server sshd[22346]: pam_unix(sshd:auth): authentication success; logname=user uid=1000 euid=1000 tty=ssh ruser= rhost=192.168.1.10  user=alice\nFeb 15 10:10:13 server sshd[22346]: Accepted password for alice from 192.168.1.10 port 51051 ssh2\nFeb 15 10:10:14 server sshd[22347]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=192.168.1.10  user=bob\n```', '```\n---\nPath: /var/log/auth.lo?\nDescription: Authentication log file with one character wildcard\nFile: \nFeb 15 16:45:10 localhost sshd[26568]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=

In [67]:
scores = []

for x in resp.completions:
    score = rate_response(sample_files_dataset[1], x)
    scores.append(score)

In [68]:
print(scores)

[MetricOutput(score=3.0), MetricOutput(score=3.0)]
